In [2]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [3]:
data = pd.read_csv(path+'data/ml_input.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkregressor.csv')

In [4]:
N = 2021

In [5]:
params_to_drop = ['season', 'round', 'driver', 'constructor', 'circuit_id', 'podium', 'driver_points_from']

In [6]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [7]:
print(*data.columns)

season round podium driver constructor circuit_id driver_points_from qualifying_pos starting_grid stage_q3 driver_points_per constructor_points_per constructor_points_before driver_points_before driver_wins_before constructor_wins_before constructor_standings_before driver_standings_before driver_last_3 constructor_last_3


In [8]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        # Scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # Make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        winners += list(prediction_df.predicted_winner.values)
        points += list(prediction_df.predicted_points.values)
        

        # # Retrain model
        # X_train = pd.concat([X_train, X_test])
        # y_train = np.append(y_train, y_test)


        # model.fit(X_train, y_train)

    return points, winners

In [9]:
params = parameters.iloc[0]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                           959
model                       nn_regressor
hidden_layer_sizes    (200, 100, 50, 25)
activation                          tanh
solver                              adam
alpha                               10.0
score                           0.736842
Name: 0, dtype: object

In [10]:
test = df[(df.season == N)]

model = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, 
    alpha=alpha, 
    random_state=1)
    
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [11]:
rounds = test['round'].nunique()

print(f'Outright Winner Accuracy: { round(test.query("predicted_winner == 1 & podium == 1").shape[0] / rounds, 2) }')

print(f'Top-Two Accuracy: { round(test.query("predicted_winner == 1 & podium <= 2").shape[0] / rounds, 2) }')

print(f'Podium Accuracy: { round(test.query("predicted_winner == 1 & podium <= 3").shape[0] / rounds, 2) }')

Outright Winner Accuracy: 0.74
Top-Two Accuracy: 0.95
Podium Accuracy: 0.95


In [12]:
def highlight_row(row):
    if row['podium'] == 1:
        return ['background-color: limegreen; opacity: 1'] * len(row)
    elif 1 < row['podium'] <= 3:
        return ['background-color: yellow; opacity: 1; color: black'] * len(row)
    elif 3 < row['podium'] <= 10:
        return ['background-color: orange'] * len(row)
    elif row['podium'] > 10:
        return ['background-color: #E34234'] * len(row)

In [41]:
q = test.query('round == 1')[['circuit_id', 'driver', 'podium', 'starting_grid', 'predicted_points']].sort_values('predicted_points', ascending=False)
q.style.set_properties(color='white').apply(highlight_row, axis=1)

,circuit_id,driver,podium,starting_grid,predicted_points
2765,bahrain,max_verstappen,2,1,15.139174
2764,bahrain,lewis_hamilton,1,2,14.216139
2766,bahrain,valtteri_bottas,3,3,12.805998
2769,bahrain,charles_leclerc,6,4,10.925527
2780,bahrain,pierre_gasly,17,5,8.877408
2770,bahrain,daniel_ricciardo,7,6,7.012418
2767,bahrain,lando_norris,4,7,5.472932
2771,bahrain,carlos_sainz,8,8,4.256889
2782,bahrain,fernando_alonso,19,9,3.320675
2768,bahrain,sergio_perez,5,11,2.731360
